[PyCaret Installation Documentation](https://pycaret.readthedocs.io/en/latest/installation.html)

In [1]:
import pandas as pd
import numpy as np
import os

from pycaret.classification import load_model, predict_model

%load_ext autoreload
%autoreload 2

# Example data

## Download glioma TCGA data from UCSC Xena
Thresholded GISTIC 2.0 gene-level SCNA data for the TCGA-LGG and TCGA-GBM can be found [here](https://xenabrowser.net/datapages/?dataset=TCGA.GBMLGG.sampleMap%2FGistic2_CopyNumber_Gistic2_all_thresholded.by_genes&host=https%3A%2F%2Ftcga.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443).

In [2]:
# Somatic copy number alteration (SCNA) data
!wget -P ../data/ https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.GBMLGG.sampleMap%2FGistic2_CopyNumber_Gistic2_all_thresholded.by_genes.gz

--2021-12-30 09:37:13--  https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.GBMLGG.sampleMap%2FGistic2_CopyNumber_Gistic2_all_thresholded.by_genes.gz
Resolving tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)... 52.216.30.120
Connecting to tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)|52.216.30.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1804219 (1.7M) [binary/octet-stream]
Saving to: ‘../data/TCGA.GBMLGG.sampleMap%2FGistic2_CopyNumber_Gistic2_all_thresholded.by_genes.gz.2’

TCGA.GBMLGG.sampleM 100%[===================>]   1.72M  4.47MB/s    in 0.4s    

2021-12-30 09:37:15 (4.47 MB/s) - ‘../data/TCGA.GBMLGG.sampleMap%2FGistic2_CopyNumber_Gistic2_all_thresholded.by_genes.gz.2’ saved [1804219/1804219]



## Load data

In [3]:
# load gene location data and exclude sex chromosomes and small chromosome arms (21p, 22p)
gene_loc_df = pd.read_csv('../data/gistic_cytoband_chr_arm_23109x4.csv', index_col=0)
chr_arms = ['1p', '1q', '2p', '2q', '3p', '3q', '4p', '4q', '5p', '5q', '6p', '6q', '7p', '7q', 
            '8p', '8q', '9p', '9q', '10p', '10q', '11p', '11q', '12p', '12q', '13q', '14q', '15q', 
            '16p', '16q', '17p', '17q', '18p', '18q', '19p', '19q', '20p', '20q', '21q', '22q']
gene_loc_df = gene_loc_df.loc[gene_loc_df['chr_arm'].isin(chr_arms)]

# load downloaded scna data
scna_filepath = '../data/TCGA.GBMLGG.sampleMap%2FGistic2_CopyNumber_Gistic2_all_thresholded.by_genes.gz'
scna_df = pd.read_csv(scna_filepath, sep='\t', index_col=0).T

# find genes with chromosome arm annotations in the gene_loc file
common_genes = [x for x in scna_df.columns if x in gene_loc_df.index]

# Treat homozyous deletions as single-copy loss & high-level amplifications as low-level amplifications 
scna_df = scna_df[common_genes].replace({-2:-1, 2:1})

# average over chromosome arms
chrarm_scna_df = pd.concat([gene_loc_df['chr_arm'], scna_df.T], axis=1, join='inner').groupby('chr_arm').mean().T

# create a dataframe that only considers losses for the 1p/19q-codeletion screen
scna_loss_df = scna_df[common_genes].replace({-2:-1, 2:1})

# average over chromosome arms 
chrarm_scna_loss_df = pd.concat([gene_loc_df['chr_arm'], scna_df.replace({1:0}).T], 
                                axis=1, join='inner').groupby('chr_arm').mean().T

# Stage 1: predict 1p/19q-codeletions

In [4]:
# screen for 1p/19q-codeletion (oligodendroglioma prediction)
thresh = 0.85
oligo_pred_idxs = chrarm_scna_loss_df.loc[(chrarm_scna_loss_df['1p'] < -thresh)
                                          & (chrarm_scna_loss_df['1q'] >= -thresh)
                                          & (chrarm_scna_loss_df['19p'] >= -thresh)
                                          & (chrarm_scna_loss_df['19q'] < -thresh)].index.tolist()
astro_pred_idxs = [x for x in chrarm_scna_loss_df.index if x not in oligo_pred_idxs]

# make oligo prediction df
oligo_pred_df = pd.DataFrame(data=np.ones(len(oligo_pred_idxs))*2, index=oligo_pred_idxs, columns=['Pred'])
oligo_pred_df['Score'] = 1
oligo_pred_df = oligo_pred_df[['Score', 'Pred']]

# Stage 2: predict IDH-mutantions among predicted astrocytic tumors

In [5]:
# load classifier
idh_classifier = load_model('../models/model_10-14-2021')

Transformation Pipeline and Model Successfully Loaded


In [6]:
# SCNA data from tumors without predicted 1p/19q-codeletions
astro_chrarm_scna_df = chrarm_scna_df.drop(index=oligo_pred_idxs)

# predict IDH-mutations
astro_pred_df = predict_model(idh_classifier, data = astro_chrarm_scna_df)
astro_pred_df = astro_pred_df[['Score', 'Label']].rename(columns={'Label':'Pred'})

# Merge predictions

In [7]:
prediction_df = oligo_pred_df.append(astro_pred_df)
prediction_df['Pred'] = prediction_df['Pred'].replace({'0.0':'IDH-Wildtype Glioma', '1.0':'IDH-Mutant Astroctyoma',
                                                       2:'Oligodendroglioma'})
prediction_df = prediction_df.rename(columns={'Score':'Confidence', 'Pred':'Prediction'})

In [8]:
threshold = 0.7
low_confidence_preds_idxs = prediction_df.loc[prediction_df['Confidence'] < threshold].index
prediction_df.loc[low_confidence_preds_idxs, 'Prediction'] = 'Uncertain'

In [9]:
prediction_df['Prediction'].value_counts()

IDH-Wildtype Glioma       581
IDH-Mutant Astroctyoma    283
Oligodendroglioma         174
Uncertain                  52
Name: Prediction, dtype: int64

In [10]:
if not os.path.exists('../results/'):
    os.makedirs('../results/')
prediction_df.to_csv('../results/tcga_ucsc_glioma_prediction_results.csv')